In [83]:
from src.backend.retrieval.file_reader import read_csv
from src.backend.retrieval.pipeline import TwoStagePipeline, preprocess_text_column

ImportError: cannot import name 'read_csv' from 'src.backend.retrieval.file_reader' (C:\Users\Jacky\OneDrive\Word\Github\final-project-sweetgreen\src\backend\retrieval\file_reader.py)

In [73]:
documents = read_csv('data/processed/all_data_cleaned.csv', column_name='cleaned_text')
type(documents)
documents

NameError: name 'read_csv' is not defined

In [54]:
documents = preprocess_documents(documents)

AttributeError: 'float' object has no attribute 'lower'